<a href="https://colab.research.google.com/github/OpenXRF/lead-screening/blob/main/Figure2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 [OpenXRF_logo_red.svg](https://openxrf.org/)

# **Towards low-cost lead screening with transmission XRF**
---

This notebook reproduces the results in **Figure 2** of our paper:


>  [C. Gaßner, J. Reisewitz, J. E. Forsyth, K. Shaker, "Towards low-cost lead screening with transmission XRF" arXiv:2511.09110 (2025)](https://doi.org/10.48550/arXiv.2511.09110)


More information can be found here:

*   Project website: [openxrf.org](https://openxrf.org/)
*   GitHub repository: [github.com/OpenXRF/lead-screening](https://github.com/OpenXRF/lead-screening/)


---


In [1]:
#@title Install packages { display-mode: "form" }
# @markdown Python packages for data processing and visualization

import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from time import time

print("All packages installed successfully!")

All packages installed successfully!


In [3]:
#@title 1) Load Data
# @markdown Load simulated data and spectrum from Github

start_time = time()
print("Loading ZIP from GitHub...")

zip_url = "https://github.com/OpenXRF/lead-screening/raw/main/data/Figure2.zip"

!wget -O Figure1.zip "{zip_url}" -q

print("Extracting...")
with zipfile.ZipFile("Figure1.zip", "r") as z:
    z.extractall("Figure2_data")

print("Reading Files...")

base_dir = "Figure2_data/Figure 2"

data = pd.read_csv(os.path.join(base_dir, "data_optimal_thickness.csv"))
thickness = data["Thickness_mm"].values

Pb1000_La_s, Pb1000_Lb_s = data["Pb1000ppm_Lalpha_signal"], data["Pb1000ppm_Lbeta_signal"]
Pb1000_La_b, Pb1000_Lb_b = data["Pb1000ppm_Lalpha_background"], data["Pb1000ppm_Lbeta_background"]

Pb100_La_s, Pb100_Lb_s = data["Pb100ppm_Lalpha_signal"], data["Pb100ppm_Lbeta_signal"]
Pb100_La_b, Pb100_Lb_b = data["Pb100ppm_Lalpha_background"], data["Pb100ppm_Lbeta_background"]

Pb10_La_s, Pb10_Lb_s = data["Pb10ppm_Lalpha_signal"], data["Pb10ppm_Lbeta_signal"]
Pb10_La_b, Pb10_Lb_b = data["Pb10ppm_Lalpha_background"], data["Pb10ppm_Lbeta_background"]

print(f"Done reading files (took {time() - start_time:.1f} s)")

Loading ZIP from GitHub...
Extracting...
Reading Files...
Done reading files (took 0.7 s)


# **Figure S1:** Optimal thickness analysis


---

-> Also FIgure 2 c and d

- 10h exposure time
- 100ppm lead in soil

In [14]:
#@title 2) Analyse the data and visualize results { display-mode: "form" }
def snr(signal, background):
    signal = np.array(signal)
    background = np.array(background)
    return np.divide(signal, np.sqrt(signal + background),
                     out=np.zeros_like(signal, dtype=float),
                     where=(signal + background) > 0)
print("\n Callculating SNR...")
# Einzel-SNRs
Pb1000_La_snr = snr(Pb1000_La_s, Pb1000_La_b)
Pb1000_Lb_snr = snr(Pb1000_Lb_s, Pb1000_Lb_b)
Pb100_La_snr  = snr(Pb100_La_s,  Pb100_La_b)
Pb100_Lb_snr  = snr(Pb100_Lb_s,  Pb100_Lb_b)
Pb10_La_snr   = snr(Pb10_La_s,   Pb10_La_b)
Pb10_Lb_snr   = snr(Pb10_Lb_s,   Pb10_Lb_b)

def snr_combined(La_s, Lb_s, La_b, Lb_b):
    S = np.array(La_s) + np.array(Lb_s)
    B = np.array(La_b) + np.array(Lb_b)
    return snr(S, B)

Pb1000_combined = snr_combined(Pb1000_La_s, Pb1000_Lb_s, Pb1000_La_b, Pb1000_Lb_b)
Pb100_combined  = snr_combined(Pb100_La_s,  Pb100_Lb_s,  Pb100_La_b,  Pb100_Lb_b)
Pb10_combined   = snr_combined(Pb10_La_s,   Pb10_Lb_s,   Pb10_La_b,   Pb10_Lb_b)

# Packen wie vorher
conc_labels = ["1000 ppm", "100 ppm", "10 ppm"]
datasets = [
    {"La_s": Pb1000_La_s, "Lb_s": Pb1000_Lb_s, "La_b": Pb1000_La_b, "Lb_b": Pb1000_Lb_b,
     "La_snr": Pb1000_La_snr, "Lb_snr": Pb1000_Lb_snr, "SNR_comb": Pb1000_combined},
    {"La_s": Pb100_La_s, "Lb_s": Pb100_Lb_s, "La_b": Pb100_La_b, "Lb_b": Pb100_Lb_b,
     "La_snr": Pb100_La_snr, "Lb_snr": Pb100_Lb_snr, "SNR_comb": Pb100_combined},
    {"La_s": Pb10_La_s, "Lb_s": Pb10_Lb_s, "La_b": Pb10_La_b, "Lb_b": Pb10_Lb_b,
     "La_snr": Pb10_La_snr, "Lb_snr": Pb10_Lb_snr, "SNR_comb": Pb10_combined},
]

print("\n Generating visualization...")

fig = make_subplots(
    rows=3,
    cols=3,
    subplot_titles=(
        "1000 ppm", "100 ppm", "10 ppm",
        "1000 ppm", "100 ppm", "10 ppm",
        "1000 ppm", "100 ppm", "10 ppm",
    ),
    horizontal_spacing=0.07,
    vertical_spacing=0.08
)

lw = 2

def ax_id(row, col):
    return (row - 1) * 3 + col

for col, (label, dset) in enumerate(zip(conc_labels, datasets), start=1):

    # Lα background
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["La_b"],
            mode="lines",
            line=dict(color="red", dash="dot", width=lw),
            name="Lα background" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=1, col=col
    )
    # Lα signal
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["La_s"],
            mode="lines",
            line=dict(color="red", width=lw),
            name="Lα signal" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=1, col=col
    )
    # Lβ background
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["Lb_b"],
            mode="lines",
            line=dict(color="blue", dash="dot", width=lw),
            name="Lβ background" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=1, col=col
    )
    # Lβ signal
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["Lb_s"],
            mode="lines",
            line=dict(color="blue", width=lw),
            name="Lβ signal" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=1, col=col
    )

    fig.update_xaxes(range=[0, 5], row=1, col=col, title_text="Thickness (mm)")
    fig.update_yaxes(
        type="log",
        row=1, col=col,
        title_text="Photon counts" if col == 1 else None,
    )

    # Lα
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["La_snr"],
            mode="lines",
            line=dict(color="orange", width=lw),
            name="Lα SNR" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=2, col=col
    )
    # Lβ
    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["Lb_snr"],
            mode="lines",
            line=dict(color="green", width=lw),
            name="Lβ SNR" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=2, col=col
    )

    # vertikale Linien für Maxima
    idx1 = np.argmax(dset["La_snr"])
    idx2 = np.argmax(dset["Lb_snr"])
    fig.add_vline(
        x=thickness[idx1],
        line=dict(color="orange", dash="dash", width=1),
        row=2, col=col,
    )
    fig.add_vline(
        x=thickness[idx2],
        line=dict(color="green", dash="dash", width=1),
        row=2, col=col,
    )

    fig.update_xaxes(range=[0, 5], row=2, col=col, title_text="Thickness (mm)")
    fig.update_yaxes(title_text="SNR" if col == 1 else None, row=2, col=col)

    fig.add_trace(
        go.Scatter(
            x=thickness,
            y=dset["SNR_comb"],
            mode="lines",
            line=dict(color="magenta", width=lw),
            name="Lα + Lβ" if col == 1 else None,
            showlegend=(col == 1),
        ),
        row=3, col=col
    )

    idx = np.argmax(dset["SNR_comb"])
    fig.add_vline(
        x=thickness[idx],
        line=dict(color="magenta", dash="dash", width=1),
        row=3, col=col,
    )

    fig.update_xaxes(range=[0, 5], row=3, col=col, title_text="Thickness (mm)")
    fig.update_yaxes(title_text="SNR" if col == 1 else None, row=3, col=col)


fig.update_layout(
    height=900,
    width=900,
    title=(
        "Optimal soil sample thickness investigation<br>"
        "Am-241 source (995 ph/s/sr), cone angle = 45°<br>"
        "N = 10⁸ ph, total time = 600 min"
    ),
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        orientation="v",
        valign="top",
        itemwidth=30,
    ),
    font=dict(size=10),
    margin=dict(t=120, l=60, r=20, b=50),
)

fig.show()



 Callculating SNR...

 Generating visualization...
